# HERE TEST USING THE 'RESULTS' OBJECT FROM SAMPLE DATA JSON

### NOTE: ignore the first object in json i.e 'final results.json' as its the city itself

### Imports 

In [38]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from dotenv import load_dotenv
import os
import json

### Load env file with api key

In [ ]:
load_dotenv() # reading all vaiables from .env file (api key)

### Load llm and test it 

In [45]:
OPENAPI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [46]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAPI_API_KEY, temperature=0.2, max_tokens=2048)

In [ ]:
llm.invoke("if active respond with active")

### Load JSON and print it 

In [ ]:
# Open and read the JSON file
with open('./final_results.json', 'r') as file:
    results_data = json.load(file)
# Print the data
print(results_data)

### Get Test Objects and Properties From json 

In [ ]:
place = results_data[1]  # first location 
location = place['geometry']['location']
print("place data")
print(place)
print("location data")
print(location)
lat, lon = location['lat'], location['lng']
print(f"Latitude: {lat}, Longitude: {lon}")

### Iterate over objects and get their attributes

In [ ]:
print("Total places:", len(results_data) - 1)
results_data.pop(0)
counter = 0
formatted_places = {}

for place in results_data:
    name = place['name']
    formatted_places[counter] = name
    print(f"Key: {counter}, Value: {formatted_places[counter]}")
    counter += 1


### Preparing The LLM promt 

In [51]:
usermood = "happy"
userhobby = "Sports"
useractivities = "Eating"

In [ ]:
prompt_template = """
You are a place selector your job is to find a place for the attributes of the user which are as follows:
Users Mood: {mood}, Users Hobbies: {hobby}, Users Activities: {activities}

You are also given a key value pair as follows: Key = place number and Value = place name. 
Your job is too find the best place for the user based on the given attributes.
you do this by looking at the name of the place which is the value.
When you find the right place return its key your answer must be a single integer in the range of keys given in the problem.
YOU MUST RETURN A VALID INTEGER AND NOTHING ELSE, EVEN IF THE USER HAS NO MATCHING PLACE RETURN YOU BEST GUESS.
Here is the list of places:
{places}
"""
prompt = ChatPromptTemplate.from_template(prompt_template) # creating the prompt using the chat prompt template library
final_prompt = prompt.format(mood = usermood, hobby = userhobby, activities = useractivities, places = formatted_places) # passing in the context and question to the prompt
print(final_prompt)

### Invoke LLM with final promt with data

In [ ]:
result = llm.invoke(final_prompt)
place_number = result.content
print(place_number)